In [2]:
import pyspark
spark = pyspark.sql.SparkSession.builder.appName("MyApp") \
            .master("spark://workspace:7077") \
            .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:1.0.2") \
            .getOrCreate()

In [3]:
import synapse.ml

In [5]:
from pyspark.sql.functions import expr, explode

weather_dir = "/home/hooniegit/git/study/rdd-manufacture/data/json_weather.json"
df = spark.read.option("multiline", "true").json(f"file://{weather_dir}") \
                      .withColumn("rows", explode(expr("response.body.items.item"))) \
                      .select("rows.*")
df.show()

23/12/26 15:23:01 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+---------+---------+--------+-------+--------+--------+--------+--------+--------+------+------+-----+------+-----+------+-----+-----+-----+------+----------+-----+---------+----------+--------------+--------+------------+----------+--------------------------+--------+------------+----------+------+---------+-----+---------+-----+-----+---------+-------+---------+-------------+------+---------+------+----------+-----+---------+-----+-----+-----+-----+-----+-----------+---------+------+--------+-----+--------+--------+-------+----------+
|avgCm10Te|avgCm20Te|avgCm30Te|avgCm5Te|avgLmac|avgM05Te|avgM10Te|avgM15Te|avgM30Te|avgM50Te| avgPa| avgPs|avgPv|avgRhm|avgTa|avgTca|avgTd|avgTs|avgWs|ddMefs|ddMefsHrmt|ddMes|ddMesHrmt|hr1MaxIcsr|hr1MaxIcsrHrmt|hr1MaxRn|hr1MaxRnHrmt|hr24SumRws|                      iscs|maxInsWs|maxInsWsHrmt|maxInsWsWd| maxPs|maxPsHrmt|maxTa|maxTaHrmt|maxWd|maxWs|maxWsHrmt|maxWsWd|mi10MaxRn|mi10MaxRnHrmt| minPs|minPsHrmt|minRhm|minRhmHrmt|minTa|minTaHrmt|minT

In [7]:
train, test = df.randomSplit([0.85, 0.15], seed=1)

In [8]:
from pyspark.ml.feature import VectorAssembler

feature_cols = df.columns[1:]
featurizer = VectorAssembler(inputCols=feature_cols, outputCol="features")
train_data = featurizer.transform(train)["avgM15Te", "features"]
test_data = featurizer.transform(test)["avgM15Te", "features"]

ModuleNotFoundError: No module named 'distutils'

In [9]:
spark.stop()